In [275]:
#coding:utf-8
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from numpy import nan

In [323]:
def pre_processing(filename):
    file_2g = pd.read_csv(filename).loc[:,['MRTime','RNCID_1', 'CellID_1','EcNo_1', 'RSCP_1', 'RNCID_2', 'CellID_2','EcNo_2', 'RSCP_2','RNCID_3', 'CellID_3','EcNo_3', 'RSCP_3', 'RNCID_4', 'CellID_4','EcNo_4', 'RSCP_4', 'RNCID_5', 'CellID_5','EcNo_5', 'RSCP_5', 'RNCID_6', 'CellID_6','EcNo_6', 'RSCP_6','Grid_ID']]
    gong_can = pd.read_csv("final_2g_gongcan.csv", encoding='gbk').loc[:,['CGI', 'LAC', 'CI', u'经度', u'纬度']]
    # merge_data = pd.merge(file_2g, gong_can, left_on=['RNCID_1','CellID_1'], right_on=['LAC','CI'], how='inner')[['MRTime','EcNo_1', 'RSCP_1', 'CGI', u'经度', u'纬度', 'Grid_ID']]

    for i in range(6):
        RNCID = "RNCID_" + str(i+1)
        CellID = "CellID_" + str(i+1)
        station = pd.merge(file_2g, gong_can, left_on=[RNCID, CellID], right_on=['LAC','CI'], how='inner')[['CGI']]
        file_2g[[str(i+1)+'station']] = station
        file_2g['RSSI'+str(i+1)] = file_2g['EcNo_' + str(i+1)] - file_2g['RSCP_' + str(i+1)]

    merge_data = file_2g
    
    merge_data['MRTime'] = merge_data['MRTime'].str.split()
    timeCol = pd.Series(map(lambda x:x[1],merge_data['MRTime']),index=merge_data.index)
    timeCol = pd.to_datetime(timeCol,format="%H:%M:%S.%f")
    merge_data['MRTime'] = timeCol
    
    merge_data = merge_data.sort_values(by=['MRTime'])
    
    for i in range(1,7):
        del merge_data['EcNo_' + str(i)],merge_data['RSCP_' + str(i)]
        
    for i in range(1,7):
        del merge_data['RNCID_' + str(i)],merge_data['CellID_' + str(i)]
        
    return merge_data

In [366]:
train_data = pre_processing('final_2g_tr.csv')
test_data = pre_processing('final_2g_te.csv')

In [433]:
train_data.head()

,MRTime,Grid_ID,1station,RSSI1,2station,RSSI2,3station,RSSI3,4station,RSSI4,5station,RSSI5,6station,RSSI6
27849,1900-01-01 00:00:00.000,159.0,460-00-6290-50305,63.0,460-00-6227-26465,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
27850,1900-01-01 00:00:00.000,159.0,460-00-6290-50305,63.0,460-00-6227-26465,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
27852,1900-01-01 00:00:00.200,159.0,460-00-6290-50305,63.0,460-00-6227-26465,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
27856,1900-01-01 00:00:00.200,159.0,460-00-6290-50305,65.0,460-00-6227-26465,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
27855,1900-01-01 00:00:00.200,159.0,460-00-6290-50305,65.0,460-00-6227-26465,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [383]:
max_rssi = train_data['RSSI1'].max()

In [384]:
max_rssi

91.0

In [431]:
def emi_score(f1,group,ru,rssi_num,dRmax):
    for i in range(1,rssi_num + 1):
        station = str(i) + 'station'
        compared = group[group[station] == f1[station]]
        compared['sum'] = 0
        compared
    
    sum = 0
    M = 0
    max_value = -1
    for row in group.index:
        f2 = group.loc[row]
        for i in range(1,rssi_num + 1):
            if(f1[str(i) + 'station'] == f2[str(i) + 'station']):
                sum += np.square(f1['RSSI' + str(i)] - f2['RSSI' + str(i)])
                M += 1
        
        if M > 0:
            df1f2 = np.sqrt(sum) / M
            temp_arr = [M*ru + dRmax - df1f2,max_value]
            try:
                max_value = np.max(temp_arr)
            except Exception, e:
                print temp_arr
    return max_value

In [389]:
emi = pd.DataFrame(0,index = test_data.index,columns=sorted(train_data['Grid_ID'].unique()))

In [400]:
emi.head()

,0.0,3.0,10.0,12.0,17.0,21.0,26.0,27.0,29.0,35.0,...,539.0,541.0,543.0,544.0,545.0,547.0,549.0,550.0,551.0,553.0
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [391]:
train_group = train_data.groupby(['Grid_ID'])

In [404]:
emi.iloc[2,12]

0

In [432]:
for row in range(0,len(test_data.head(20))):
    fnew = test_data.loc[row]
    for key,group in train_group:
        emi[int(row),int(key)] = emi_score(fnew,group,1,6,max_rssi)

MemoryError: 

In [395]:
type(train_group.groups)

dict

In [ ]:
emi